In [1]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
from torchvision import models
import torch.optim as optim
from torchvision import datasets, models, transforms
import numpy as np
from tqdm import tqdm

In [2]:
# Đường dẫn đến dataset
DATASET_PATH = '/kaggle/input/hwd-dataset/digits_data_final'
TRAIN_DIR = os.path.join(DATASET_PATH, 'train')
VAL_DIR = os.path.join(DATASET_PATH, 'val')

# Tham số
BATCH_SIZE = 8
IMG_SIZE = 224
NUM_CLASSES = 10 
EPOCHS = 10
num_workers = os.cpu_count()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Dataset**

In [3]:
!pip install pillow-heif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00


In [4]:
def count_images_in_folder(folder_path):
    total = 0
    class_folders = glob.glob(os.path.join(folder_path, "*/"))
    for class_path in class_folders:
        image_files = glob.glob(os.path.join(class_path, "*"))
        total += len(image_files)
    return total

# Tổng số ảnh trong train
total_train = count_images_in_folder(f"{DATASET_PATH}/train")
print(f"Tổng số ảnh trong TRAIN: {total_train}")

# Tổng số ảnh trong val
total_val = count_images_in_folder(f"{DATASET_PATH}/val")
print(f"Tổng số ảnh trong VAL: {total_val}")

Tổng số ảnh trong TRAIN: 5712
Tổng số ảnh trong VAL: 1433


In [5]:
from pillow_heif import register_heif_opener

class custom_image_dataset(Dataset):
    """
    Một Dataset tùy chỉnh đa năng cho cả train/val và test.

    - Nếu test=False: Quét các thư mục con làm nhãn.
    - Nếu test=True: Quét tất cả ảnh trong thư mục gốc và gán nhãn là -1.
    """
    def __init__(self, root_dir, transform=None, test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.test = test
        # SỬA LỖI 2: Thống nhất dùng tên self.image_paths
        self.image_paths = []
        self.labels = []

        if not os.path.isdir(root_dir):
            raise ValueError(f"Đường dẫn không tồn tại: {root_dir}")

        candidate_files = []
        if not self.test:
            # --- Chế độ TRAIN/VAL ---
            class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
            class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}
            print(f"Chế độ TRAIN/VAL. Đã tìm thấy các lớp: {class_names} tại '{root_dir}'")

            for class_name in class_names:
                class_dir = os.path.join(root_dir, class_name)
                label = class_to_idx[class_name]
                for filename in os.listdir(class_dir):
                    if filename.lower().endswith('.md'):
                        print('Found MarkDown')
                        pass
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.heic', '.heif', '.jfif')):
                        candidate_files.append((os.path.join(class_dir, filename), label))
        else:
            # --- Chế độ TEST ---
            print(f"Chế độ TEST. Đang quét tất cả ảnh trong '{root_dir}'...")
            for filename in os.listdir(root_dir):
                if filename.lower().endswith('.md'):
                        print('Found MarkDown')
                        pass
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.heic', '.heif', '.jfif')):
                    # SỬA LỖI 1: Dùng root_dir thay vì class_dir
                    full_path = os.path.join(root_dir, filename)
                    candidate_files.append((full_path, -1))

        # Xác thực các file ứng viên
        print(f"Đã tìm thấy {len(candidate_files)} file ứng viên. Bắt đầu xác thực...")
        corrupted_files = []
        for img_path, label in tqdm(candidate_files, desc="Đang xác thực file"):
            try:
                with Image.open(img_path) as img:
                    img.verify()
                # Nếu file hợp lệ, thêm vào danh sách cuối cùng
                self.image_paths.append(img_path)
                self.labels.append(label)
            except Exception:
                corrupted_files.append(img_path)
        
        print("\n--- Hoàn thành quét và xác thực ---")
        print(f"Tổng số ảnh hợp lệ có thể sử dụng: {len(self.image_paths)}")
        if corrupted_files:
            print(f"Đã phát hiện và loại bỏ {len(corrupted_files)} file bị lỗi.")

    def __len__(self):
        # SỬA LỖI 2: Dùng đúng tên biến
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
            
        return image, label, img_path

In [6]:
register_heif_opener()

In [7]:
# Định nghĩa các phép biến đổi cho dữ liệu
# Rất quan trọng: phải chuẩn hóa giống như khi pre-train mô hình
data_transforms = {
    'train': transforms.Compose([
        transforms.Lambda(lambda img: img.convert('RGB')),
        transforms.Resize((224, 224)),
        transforms.RandomRotation(10),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Lambda(lambda img: img.convert('RGB')),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ]),
}

try:
    image_datasets = {
    'train': custom_image_dataset(TRAIN_DIR, transform=data_transforms['train']),
    'val': custom_image_dataset(VAL_DIR, transform=data_transforms['val'])
}


    dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
}

except ValueError as e:
    print(e)
except Exception as e:
    print(f"Đã xảy ra lỗi không mong muốn: {e}")

Chế độ TRAIN/VAL. Đã tìm thấy các lớp: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] tại '/kaggle/input/hwd-dataset/digits_data_final/train'
Đã tìm thấy 5712 file ứng viên. Bắt đầu xác thực...


Đang xác thực file: 100%|██████████| 5712/5712 [00:50<00:00, 112.21it/s]



--- Hoàn thành quét và xác thực ---
Tổng số ảnh hợp lệ có thể sử dụng: 5712
Chế độ TRAIN/VAL. Đã tìm thấy các lớp: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] tại '/kaggle/input/hwd-dataset/digits_data_final/val'
Đã tìm thấy 1433 file ứng viên. Bắt đầu xác thực...


Đang xác thực file: 100%|██████████| 1433/1433 [00:12<00:00, 116.74it/s]


--- Hoàn thành quét và xác thực ---
Tổng số ảnh hợp lệ có thể sử dụng: 1433


In [8]:
model = models.efficientnet_b0(weights="EfficientNet_B0_Weights.DEFAULT")

# Đóng băng tất cả các tham số của mô hình
for param in model.parameters():
    param.requires_grad = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.efficientnet_b0(weights="EfficientNet_B0_Weights.DEFAULT")
in_feats = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_feats, 10)
)
model = model.to(device)


# In ra cấu trúc classifier mới
print(model.classifier)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 151MB/s]


Sequential(
  (0): Dropout(p=0.3, inplace=False)
  (1): Linear(in_features=1280, out_features=10, bias=True)
)


In [9]:
import time
# --- Cấu hình Logging ---
LOG_FILE = 'log_train_baseline.txt'

# --- Mở file log để ghi ---
with open(LOG_FILE, 'w') as log_file:
    log_file.write('Epoch,Train Loss,Train Acc,Val Loss,Val Acc,Time\n')


# --- Bắt đầu Vòng lặp Huấn luyện chính ---
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
start_time_total = time.time()

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    print(f'Epoch {epoch+1}/{EPOCHS}')
    print('-' * 10)

    # Mỗi epoch có 2 pha: training và validation
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        # Lặp qua dữ liệu
        for inputs, labels, _ in tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Phase"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Lưu kết quả vào history
        if phase == 'train':
            history['train_loss'].append(epoch_loss)
            history['train_acc'].append(epoch_acc.item())
        else:
            history['val_loss'].append(epoch_loss)
            history['val_acc'].append(epoch_acc.item())
    
    # --- Ghi Log sau mỗi epoch ---
    epoch_time = time.time() - epoch_start_time
    with open(LOG_FILE, 'a') as log_file:
        log_line = (f"{epoch+1},{history['train_loss'][-1]:.4f},{history['train_acc'][-1]:.4f},"
                    f"{history['val_loss'][-1]:.4f},{history['val_acc'][-1]:.4f},{epoch_time:.2f}s\n")
        log_file.write(log_line)
    
total_training_time = time.time() - start_time_total
print(f'\nHoàn thành huấn luyện! Tổng thời gian: {total_training_time // 60:.0f}m {total_training_time % 60:.0f}s')


# --- ĐÁNH GIÁ TRÊN MÔ HÌNH SAU KHI KẾT THÚC HUẤN LUYỆN ---
print("\n--- Final evaluation on the model from the last epoch ---")
model.eval() 

final_val_loss = 0.0
final_val_corrects = 0

with torch.no_grad():
    for inputs, labels, _ in tqdm(dataloaders['val'], desc="Final Evaluation"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        final_val_loss += loss.item() * inputs.size(0)
        final_val_corrects += torch.sum(preds == labels.data)

final_loss = final_val_loss / len(dataloaders['val'].dataset)
final_acc = final_val_corrects.double() / len(dataloaders['val'].dataset)

print("\n--- Final Evaluation Results (on last epoch's model) ---")
print(f"Validation Loss: {final_loss:.4f}")
print(f"Validation Accuracy: {final_acc:.4f}")

with open(LOG_FILE, 'a') as log_file:
    log_file.write("\n--- Final Evaluation Results (on last epoch's model) ---\n")
    log_file.write(f"Validation Loss: {final_loss:.4f}\n")
    log_file.write(f"Validation Accuracy: {final_acc:.4f}\n")

Epoch 1/10
----------


Train Phase: 100%|██████████| 714/714 [02:19<00:00,  5.11it/s]


Train Loss: 0.9520 Acc: 0.6957


Val Phase: 100%|██████████| 180/180 [00:32<00:00,  5.46it/s]


Val Loss: 0.4438 Acc: 0.8842
Epoch 2/10
----------


...

Train Loss: 0.1761 Acc: 0.9459


Val Phase: 100%|██████████| 180/180 [00:31<00:00,  5.72it/s]


Val Loss: 0.2447 Acc: 0.9386

Hoàn thành huấn luyện! Tổng thời gian: 26m 59s

--- Final evaluation on the model from the last epoch ---


Final Evaluation: 100%|██████████| 180/180 [00:30<00:00,  5.90it/s]


--- Final Evaluation Results (on last epoch's model) ---
Validation Loss: 0.2447
Validation Accuracy: 0.9386


# **Predict 2K**

In [10]:
test_dir = '/kaggle/input/hand-written-ditgit'
test_list = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]

print(f"Số lượng file test: {len(test_list)}")

test_dataset = custom_image_dataset(test_dir, transform = data_transforms['val'], test=True )

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=2)

Số lượng file test: 2939
Chế độ TEST. Đang quét tất cả ảnh trong '/kaggle/input/hand-written-ditgit'...
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Đã tìm thấy 2928 file ứng viên. Bắt đầu xác thực...


Đang xác thực file: 100%|██████████| 2928/2928 [00:24<00:00, 121.27it/s]


--- Hoàn thành quét và xác thực ---
Tổng số ảnh hợp lệ có thể sử dụng: 2928


In [11]:
predict_txt = ""
with torch.no_grad():
    for data in tqdm(test_loader, desc="Đang dự đoán:....."):
        images, labels, paths = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # save
        for path, pred in zip(paths, predicted):
            path = path.replace(test_dir, "").lstrip(os.sep)  
            predict_txt += f"{path},{pred.item()}\n"

# Write to file in text mode
with open("/kaggle/working/predict_2k.txt", "w") as file:
    file.write(predict_txt)
print("Predictions saved in 'predict_2k.txt'")

Đang dự đoán:.....: 100%|██████████| 366/366 [00:41<00:00,  8.80it/s]

Predictions saved in 'predict_2k.txt'


# **Predict 10k**

In [12]:
test_dir = '/kaggle/input/data-10k'
test_list = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]

print(f"Số lượng file test: {len(test_list)}")

test_dataset = custom_image_dataset(test_dir, transform = data_transforms['val'], test=True )

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=2)

Số lượng file test: 9998
Chế độ TEST. Đang quét tất cả ảnh trong '/kaggle/input/data-10k'...
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Found MarkDown
Đã tìm thấy 9987 file ứng viên. Bắt đầu xác thực...


Đang xác thực file: 100%|██████████| 9987/9987 [01:38<00:00, 101.30it/s]


--- Hoàn thành quét và xác thực ---
Tổng số ảnh hợp lệ có thể sử dụng: 9975
Đã phát hiện và loại bỏ 12 file bị lỗi.


In [13]:
predict_txt = ""
with torch.no_grad():
    for data in tqdm(test_loader, desc="Đang dự đoán:...."):
        images, labels, paths = data
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # save
        for path, pred in zip(paths, predicted):
            path = path.replace(test_dir, "").lstrip(os.sep)  
            predict_txt += f"{path},{pred.item()}\n"

# Write to file in text mode
with open("/kaggle/working/predict_10k.txt", "w") as file:
    file.write(predict_txt)
print("Predictions saved in 'predict_10k.txt'")

Đang dự đoán:....: 100%|██████████| 1247/1247 [03:16<00:00,  6.36it/s]

Predictions saved in 'predict_10k.txt'
